In [ ]:
from tkinter import *
import os
from pytube import YouTube
import moviepy.editor as mp
from tkinter.filedialog import askopenfilename
from pathlib import Path
import import_ipynb

In [ ]:
# Se importa el archivo
import word_Segmentation

In [ ]:
# Titulo del video de Youtube
titulo_video = ""
# Titulo del video de Youtube sin caracteres especiales
nuevo_titulo = ""

archivo_selecionado = ""

# Obtiene el directorio en el que te encuentras
current_directory = os.getcwd()
print("currentdirectory:" + current_directory)

# Crea una carpeta llamada recetas
final_directory = os.path.join(current_directory, r'recetas')
print(final_directory)
# Si la carpeta no existe la crea, sino no hace nada
if not os.path.exists(final_directory):
    os.makedirs(final_directory)
    # Comprueba que dentro de recetas no exista la carpeta transcripciones, si no existe la crea
    directorio_txt = os.path.join(final_directory, r'transcripciones')
    if not os.path.exists(directorio_txt):
        os.makedirs(directorio_txt)


# Funcion que descarga el video en el directorio en el que te encuentres
def download():

    try:
        # Recoge el link y lo localiza en Youtube
        url = YouTube(str(link.get()))
        # Obtiene el titulo del video
        titulo_video = url.title
        # Elimina los caracteres especiales del titulo del video
        nuevo_titulo = titulo_video.replace(',', '').replace('.', '').replace('!', '').replace('¡', '').replace('?',
                                                                                                                '').replace(
            '¿', '').replace('(', '').replace(')', '')
        # Obtiene el video con la mejor resolucion
        video = url.streams.get_highest_resolution()
        # Obtiene el path en el que te encuentras
        ruta = os.getcwd()
        print("ruta: " + ruta)
        # Descarga el video en el directorio de recetas
        video.download(ruta + "\\recetas")
        # Una vez se ha completado la descarga despliega un mensaje de "descargado"
        Label(root, text="Descarga completada", font="arial 15").place(x=100, y=120)

    except Exception as e:
        print("Error a la hora de descargar el vídeo deseado")


def open_file():

    try:
        # Se pide al usuario que seleccione un archivo
        doc_selected = askopenfilename()
        
        if not doc_selected:
            print("No se ha seleccionado ningun archivo")
        else:
        
            if doc_selected.endswith(".mp4"):
                print("Doc_selected:" + doc_selected)
                # Se cambia el directorio a la carpeta de recetas
                os.chdir(final_directory)
                recetas = os.getcwd()
                print("openfile: "+ recetas)

                # Se obtiene el nombre del archivo seleccionado
                archivo_selecionado = Path(doc_selected).stem

                # Se extrae del video el audio
                clip = mp.VideoFileClip(doc_selected)
                clip.audio.write_audiofile(recetas + '\\' + archivo_selecionado + '.wav')
                # Se fragmenta el audio y se crea el fichero .txt
                word_Segmentation.fragmentar_audio(archivo_selecionado)
                Label(root, text="Terminado", font="arial 15").place(x=100, y=120)
                # Se vuelve al directorio principal
                os.chdir(current_directory)
            else:
                print("Formato no aceptable")
                Label(root, text="Fomato de vídeo no válido", font="arial 15").place(x=100, y=120)
                os.chdir(current_directory)
                
    except Exception as e:
        print("El archivo que busca no existe.")
        os.chdir(current_directory)


root = Tk()
root.geometry("500x300")
# Hace que la ventana se pueda ajustar
root.resizable(0, 0)
root.title('Descargador de recetas')

Label(root, text="Descarga de vídeos", font='san-serif 14 bold').pack()
link = StringVar()  # Specifying the variable type
Label(root, text="Pega tu link aquí", font='san-serif 15 bold').place(x=150, y=55)
link_enter = Entry(root, width=70, textvariable=link).place(x=30, y=85)

# Cuando se pulsa sobre el boton la funcion download se ejecuta
Button(root, text='Descargar', font='san-serif 16 bold', bg='red', padx=2, command=download).place(x=175, y=150)

Button(root, text='Seleccionar video', font='san-serif 16 bold', bg='blue', padx=2,
       command=open_file).place(x=135, y=200)

root.mainloop()